In [ ]:
import os
import pandas as pd
import numpy as np
import time 
import pwd
import re
import io
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from scipy import stats
from scipy.stats import shapiro
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import string
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import math
import plotly.graph_objects as go
import plotly.express as px
import warnings
import boto3
import s3fs
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import ipywidgets as widgets
from ipywidgets import FileUpload
from ipywidgets import TwoByTwoLayout
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import VBox, HBox, Label, Box
from ipywidgets import Button, Layout
from pandas.errors import EmptyDataError 
#Print function
def printmd(string):
    display(Markdown(string))
np.set_printoptions(threshold=100000)
warnings.filterwarnings('ignore')

def get_file_path(req_file_name):
    for root, directory, file in os.walk(r'../'):
        for name in file:
            if name == req_file_name:
                return os.path.abspath(os.path.join(root, name))
            
def clean_data(req_df):
    req_df = req_df.dropna(axis = 1, thresh = req_df.shape[0] / 2)
    req_df = req_df.dropna(axis = 0, thresh = req_df.shape[1] / 2)
    req_df = req_df.reset_index(drop=True)
    return req_df

def read_data(req_upload):
    file_name = next(iter(req_upload.value))
    file_path = get_file_path(file_name)
    content = upload.value[file_name]['content']
    try:
        data = pd.read_csv(file_path)          
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding ='latin-1')
    data = clean_data(data)
    st = os.stat(file_path)
    return data, st, file_name, file_path

#Set the stop words
stop_words = set(stopwords.words("english"))

#Initialise the words corpus
words_corpora = set(nltk.corpus.words.words())
np.set_printoptions(threshold=sys.maxsize)

#Initialise the date words
stop_words_time = ['date', 'time', 'year', 'hour', 'o\'clock']

# Administrative metadata

##### Administrative metadata is information to help manage a resource, like resource type, permissions, and when and how it was created.

In [ ]:
def administrative_metadata(req_file_path, file_name, req_st):
    #Check for access
    read_access = os.access(file_path, os.R_OK)
    write_access = os.access(file_path, os.W_OK)
    execution_access = os.access(file_path, os.X_OK)
    existance_file = os.access(file_path, os.F_OK)    
    
    #Create dataframe
    administrative_data = pd.DataFrame(columns = ['file_name', 'file_read_access', 'file_write_access', 'file_execution_access', 'file_existance', 'file_creation_date'])

    #Set values
    administrative_data.loc[0, 'file_name'] = file_name
    administrative_data.loc[0, 'file_read_access'] = read_access
    administrative_data.loc[0, 'file_write_access'] = write_access
    administrative_data.loc[0, 'file_execution_access'] = execution_access
    administrative_data.loc[0, 'file_existance'] = existance_file
    administrative_data.loc[0, 'file_creation_date'] = time.ctime(req_st.st_ctime)
    
    return administrative_data
    

# Descriptive metadata

##### Descriptive metadata is descriptive information about a resource. It is used for discovery and identification. It includes elements such as title, abstract, author, and keywords.

In [ ]:
#Filter function to filter out time columns and columns that provide little information
def filter_columns(req_string): 
    pattern  = re.compile(r'' + "|".join(stop_words_time), re.IGNORECASE)
    is_string_column = lambda x : data[x].dtypes == 'O' or data[x].dtypes == 'S'
    return not bool(pattern.search(req_string)) and is_string_column(req_string)

#Check if a column is about time/dates
def is_time_column(req_string):
    pattern  = re.compile(r'' + "|".join(stop_words_time), re.IGNORECASE)
    return bool(pattern.search(req_string))

#Check if the words in the columns appear in the English corpus 
def is_meaningful_word(req_string):
    is_meaningful = pd.Series(list(map(lambda x: "True" if ((x != 'nan' and pd.isnull(x) != True) and str(x).lower().translate(str.maketrans('', '', string.punctuation)) in words_corpora) else "False", " ".join(map(str,req_string)).split()))).value_counts()
    return 'True' in is_meaningful.index.values and is_meaningful['True'] > 0

def data_keywords(req_data):
    column_names = req_data.columns
    df_nlp = pd.DataFrame()
    keywords = list()
    for column in filter(filter_columns, column_names):
        #Tokenize and lower case
        df_nlp[column] = req_data[column].apply(lambda x : str(x).lower().split(" "))
        vectorizer = TfidfVectorizer(stop_words="english", lowercase = False, ngram_range = (1,2))
        try:
            #Create the tfidf matrix 
            tfidf_matrix = vectorizer.fit_transform(df_nlp[column].astype(str).tolist())
            
            #Get the tokens
            feature_names = vectorizer.get_feature_names()
            
            #Get the words that appear
            feature_indexes = tfidf_matrix[:,:].nonzero()[1]
            
            #Create a DataFrame to store the tokens with their tfidf
            data_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns = vectorizer.get_feature_names())
            
            #Sort the values in descending order
            data_tfidf = data_tfidf.apply(lambda x : x.mean()).sort_values(ascending=False)            
            if len(data_tfidf) >= 3:
                #Test the 
                stat, p = shapiro(data_tfidf)
                alpha = 0.05
                if p <= alpha and is_meaningful_word(req_data[column]):
                    keywords.append((data_tfidf[data_tfidf > data_tfidf.describe()['max'] - data_tfidf.describe()['std']].index.tolist()))
        except ValueError:
            pass
    for column in column_names:
        if is_time_column(column):
            keywords.append(["time analysis"])
            break;
    return keywords

In [ ]:
def descriptive_metadata(req_file_path, req_data, req_file_name):
    st = os.stat(req_file_path)
    descriptive_data = pd.DataFrame(columns = ['file_name', 'file_author'])
    author_name = pwd.getpwuid(st.st_uid).pw_gecos
    descriptive_data.loc[0, 'file_name'] = req_file_name
    descriptive_data.loc[0, 'file_author'] = author_name
    keywords = data_keywords(req_data)
    descriptive_data.loc[0, 'keywords'] = "\n".join(map(str, keywords))  
    return descriptive_data

# Structural

###### Structural metadata is metadata about containers of data and indicates how compound objects are put together, for example, how pages are ordered to form chapters. It describes the types, versions, relationships and other characteristics of digital materials.

In [ ]:
def structural_metadata(req_file_name, req_data):
    #Create a DataFrame to store the structural metadata
    structural_data = pd.DataFrame(columns = ['column_types'])
    col_type = list()
    for column in req_data.columns:
        col_type.append(type(req_data[column][0]))
    col_type = set(col_type)
    
    #Get the information
    structural_data.loc[0, 'file_name'] = req_file_name
    structural_data.loc[0, 'column_types'] = ",".join(map(str,col_type))
    structural_data.loc[0, 'column_numbers'] = len(data.columns)
    structural_data.loc[0, 'entries'] = data.shape[0]
    return structural_data

In [ ]:
def get_meta_data(req_file_path, req_file_name, req_data, req_st):
    #Administrative metadata
    administrative_mt = administrative_metadata(req_file_path, req_file_name, req_st)
    
    #Descriptive metadata
    descriptive_mt = descriptive_metadata(req_file_path, req_data, req_file_name)
    
    #Structural metadata
    structural_mt = structural_metadata(req_file_name ,req_data)
    
    #Create metadata dataframe
    mt_data = pd.DataFrame()
    mt_data = pd.merge(administrative_mt, descriptive_mt, on=['file_name'])
    mt_data = pd.merge(mt_data, structural_mt, on=['file_name'])
    return mt_data

# Choose the file to get the metadata

# Local metadata file

Try to get a metadata file. 

In [ ]:
try:
    meta_data = pd.read_csv('../data_meta.csv')
    index_file = meta_data.shape[0] - 1
except OSError:
    meta_data = pd.DataFrame(columns=['file_name', 'file_read_access', 'file_write_access',
       'file_execution_access', 'file_existance', 'file_creation_date',
       'file_author', 'keywords', 'column_types', 'column_numbers', 'entries'])

# Single file - local 

## Select the file using File explorer

In [ ]:
upload = FileUpload(accept='.csv')
upload

In [ ]:
data, st, file_name, file_path = read_data(upload)
data, st, file_name, file_path

# Get the metadata for the selected file

In [ ]:
mt_data_current_file = get_meta_data(file_path, file_name, data, st)
meta_data = meta_data.append(mt_data_current_file)
meta_data

# Multiple files - one single local metadata file


For multiple files to get the metadata file. Change the path_data variable with the desired folder in order to select the required files.

In [ ]:
path_data = ".."
folders_files = os.walk("..")
# if ".DS_Store" in folders:
#     folders.remove(".DS_Store")
folders = []
for x in folders_files:
    folders.append(x[0])
for current_folder in tqdm(folders, desc = "Folders", leave = True):
    for file_name in os.listdir(current_folder):
        if file_name.endswith(".csv") == True:
            file_path = current_folder + "/" + file_name
            st = os.stat(file_path)
            try:
                data = pd.read_csv(file_path)          
            except UnicodeDecodeError:
                data = pd.read_csv(file_path, encoding ='latin-1')
            except EmptyDataError:
                pass
            data = clean_data(data)
            if data.empty == False:
                mt_data_current_file = get_meta_data(file_path, file_name, data, st)
                meta_data = meta_data.append(mt_data_current_file)

# Write the metadata to the local file

In [ ]:
meta_data = meta_data.reset_index()
meta_data = meta_data.drop(["index"],axis=1)
meta_data.to_csv('../data_meta.csv', index=True)

# Write the metadata file to S3

In [ ]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

## List all buckets available

In [ ]:
def list_all_buckets():
    try:
        bucket_list = [bucket.name for bucket in s3.buckets.all()]
        return bucket_list
    except Exception as e:
        print("Unexpected error: %s", e)

print('Available buckets: ' + str(list_all_buckets()))

## List all files in a bucket

In [ ]:
def list_all_buckets(req_bucket_name = "", req_bucket = None): 
    try:
        folders_bucket = list()
        objects_list = list()
        if req_bucket_name != "":
            my_bucket = s3.Bucket(req_bucket_name)
        elif req_bucket != None:
            my_bucket = req_bucket
        else:
            print("Please provide an argument")
            return None
        for object in my_bucket.objects.all():
            if(re.match(r".*\..*", object.key)):    
                object_name = object.key[object.key.rfind("/") + 1:]
                objects_list.append(object_name)
            else: 
                folders_bucket.append(object.key)
        return objects_list, folders_bucket
    except Exception as e:
        print("Unexpected error: %s" % e)
        
mybucket = s3.Bucket("uom.bioinformatics") 
bucket_files, bucket_folders = list_all_buckets(req_bucket = mybucket)

printmd("**Files in bucket " + mybucket.name + ":** \n")
for value in bucket_files:
    print(value)
    print()

## List all folders in a bucket

In [ ]:
printmd("**Folders in bucket " + mybucket.name + ":**\n")
for value in bucket_folders:
    print(value)
    print()

## Write single metadata file to a bucket folder

In [ ]:
s3_client.upload_file("../data_meta.csv", "uom.bioinformatics", "data_meta.csv")